<h1> Parameter Tunning 
    
    Look at n_neurons & n_hidden layers

# Importing libraries:

In [11]:
!pip install pandas
!pip install sklearn

import pandas as pd;
import numpy as np;
import sklearn

# Import data; set X and y; fill nan values and split in test and training  data:

In [12]:
# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# for later use
features_num = 15

# 2018 data
data = data.loc[data.index > 2018070000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values
data.fillna(method = 'ffill', inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [13]:
from sklearn.preprocessing import MinMaxScaler

# data scaling  (including offer (y))
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing packages:

In [14]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []
hist_list = []
y_pred_list = []
prediction_list = []
time_count = []

# Prepare of data according to LSTM needs,  create regressor & tune:

In [ ]:
# parameters
steps = 96
#n_hidden = 1
#units = 100
batch_size = 96

from sklearn.model_selection import ParameterGrid

# no hidden or one hidden layer
parameters = {'n_hidden': [1, 2],
              'units': [50, 100, 150, 200]}

all_param = ParameterGrid(parameters)

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# divide features and labels
X_train = data_train[:, 0:15] 
y_train = data_train[:, -1]
X_test = data_test[:, 0:15] 
y_test = data_test[:, -1] 

# divide data into train and test 
X_train, X_val, y_train, y_val = train_test_split(
         X_train, y_train, test_size = 0.15, shuffle=False)

# put data into correct shape
X_train, y_train = split_data(X_train, y_train, steps)
X_test, y_test = split_data(X_test, y_test, steps)
X_val, y_val = split_data(X_val, y_val, steps)

X_train = cut_data(X_train, batch_size)
y_train = cut_data(y_train, batch_size)
X_test = cut_data(X_test, batch_size)
y_test = cut_data(y_test, batch_size)
X_val = cut_data(X_val, batch_size)
y_val = cut_data(y_val, batch_size)

# design the LSTM
def regressor_tunning(kernel_initializer = 'he_normal',
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    if n_hidden == 0:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    else:
        model.add(LSTM(units = units,                    
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       return_sequences = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
        model.add(LSTM(units = units, 
                       batch_input_shape = (batch_size, steps, features_num), 
                       stateful = True,
                       kernel_initializer = kernel_initializer,
                       bias_initializer = bias_initializer))
        model.add(LeakyReLU(alpha = 0.2))
        model.add(Dropout(0.2))
    model.add(Dense(1, activation='linear'))
    model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = 'Adamax')
    return model

for i in range(len(all_param)):
    
    units = all_param[i]['units']
    n_hidden = all_param[i]['n_hidden']
    
    model = regressor_tunning()

    # fitting the LSTM to the training set
    history = model.fit(X_train,
                        y_train, 
                        batch_size = batch_size, 
                        epochs = 180,
                        shuffle = False, 
                        validation_data = (X_val, y_val))
    
    model = regressor_tunning()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[15]) + (sc_X.data_min_[15])
    y_test = (y_test * sc_X.data_range_[15]) + (sc_X.data_min_[15])

    y_pred_list.append(y_pred)
    
    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mse_error = mse(y_test, y_pred) # 1479.61335
    mae_error = mae(y_test, y_pred) # 23.1525
    
    rmse_gen.append(rmse_error)
    mse_gen.append(mse_error)
    mae_gen.append(mae_error)
    
    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])
    
    # create array same size as y_test
    y_spike_occ = y_spike_occ.iloc[- len(y_test):]
    y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values
    
    
    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    
    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]
    
    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mse_spike = mse(y_test_spike, y_pred_spike)
    mae_spike = mae(y_test_spike, y_pred_spike)
    
    rmse_spi.append(rmse_spike)
    mse_spi.append(mse_spike)
    mae_spi.append(mae_spike)
    
    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================
    
    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)
    
    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 
    
    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]
    
    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mse_normal = mse(y_test_normal, y_pred_normal)
    mae_normal = mae(y_test_normal, y_pred_normal)
    
    rmse_nor.append(rmse_normal)
    mse_nor.append(mse_normal)
    mae_nor.append(mae_normal)

Epoch 1/180
65/65 [==============================] - 17s 265ms/step - loss: 0.1544 - mse: 0.1544 - mae: 0.3120 - val_loss: 0.0021 - val_mse: 0.0021 - val_mae: 0.0341
Epoch 2/180
65/65 [==============================] - 14s 214ms/step - loss: 0.0982 - mse: 0.0982 - mae: 0.2502 - val_loss: 0.0027 - val_mse: 0.0027 - val_mae: 0.0398
Epoch 3/180
65/65 [==============================] - 12s 192ms/step - loss: 0.0672 - mse: 0.0672 - mae: 0.2046 - val_loss: 0.0024 - val_mse: 0.0024 - val_mae: 0.0364
Epoch 4/180
65/65 [==============================] - 11s 168ms/step - loss: 0.0448 - mse: 0.0448 - mae: 0.1679 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0321
Epoch 5/180
65/65 [==============================] - 12s 181ms/step - loss: 0.0327 - mse: 0.0327 - mae: 0.1441 - val_loss: 0.0014 - val_mse: 0.0014 - val_mae: 0.0232
Epoch 6/180
65/65 [==============================] - 12s 178ms/step - loss: 0.0235 - mse: 0.0235 - mae: 0.1222 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0232
Epoc

65/65 [==============================] - 12s 190ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0203 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0183
Epoch 51/180
65/65 [==============================] - 12s 188ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0198 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0185
Epoch 52/180
65/65 [==============================] - 12s 189ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0197 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0183
Epoch 53/180
65/65 [==============================] - 13s 194ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0198 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0182
Epoch 54/180
65/65 [==============================] - 13s 193ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0193 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.01840.01 - ETA: 3s - loss: 9.5273e-04 - mse: 9.
Epoch 55/180
65/65 [==============================] - 12s 188ms/step - loss: 0.0010 - mse: 0.0010 - mae: 0.0194 - val_loss: 0.0011 - va

65/65 [==============================] - 13s 205ms/step - loss: 9.1986e-04 - mse: 9.1986e-04 - mae: 0.0179 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0190
Epoch 97/180
65/65 [==============================] - 13s 207ms/step - loss: 9.2017e-04 - mse: 9.2017e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 98/180
65/65 [==============================] - 13s 201ms/step - loss: 9.1699e-04 - mse: 9.1699e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 99/180
65/65 [==============================] - 13s 202ms/step - loss: 9.1326e-04 - mse: 9.1326e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 100/180
65/65 [==============================] - 13s 201ms/step - loss: 9.1458e-04 - mse: 9.1458e-04 - mae: 0.0178 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 101/180
65/65 [==============================] - 14s 210ms/step - loss: 9.1392e-04 - mse: 9.1392e-04 - mae: 0.0177 - val_loss: 0.00

65/65 [==============================] - 14s 215ms/step - loss: 8.8032e-04 - mse: 8.8032e-04 - mae: 0.0173 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0183
Epoch 143/180
65/65 [==============================] - 14s 208ms/step - loss: 8.7278e-04 - mse: 8.7278e-04 - mae: 0.0172 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0182
Epoch 144/180
65/65 [==============================] - 14s 208ms/step - loss: 8.8037e-04 - mse: 8.8037e-04 - mae: 0.0173 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0184
Epoch 145/180
65/65 [==============================] - 14s 214ms/step - loss: 8.7357e-04 - mse: 8.7357e-04 - mae: 0.0172 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0185
Epoch 146/180
65/65 [==============================] - 14s 209ms/step - loss: 8.7523e-04 - mse: 8.7523e-04 - mae: 0.0172 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0183
Epoch 147/180
65/65 [==============================] - 14s 208ms/step - loss: 8.7151e-04 - mse: 8.7151e-04 - mae: 0.0173 - val_loss: 0

65/65 [==============================] - 32s 499ms/step - loss: 0.0089 - mse: 0.0089 - mae: 0.0736 - val_loss: 8.0520e-04 - val_mse: 8.0520e-04 - val_mae: 0.0170
Epoch 9/180
65/65 [==============================] - 32s 488ms/step - loss: 0.0074 - mse: 0.0074 - mae: 0.0673 - val_loss: 8.0041e-04 - val_mse: 8.0041e-04 - val_mae: 0.0158
Epoch 10/180
65/65 [==============================] - 31s 483ms/step - loss: 0.0064 - mse: 0.0064 - mae: 0.0619 - val_loss: 9.0587e-04 - val_mse: 9.0587e-04 - val_mae: 0.0223
Epoch 11/180
65/65 [==============================] - 33s 501ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0569 - val_loss: 8.1979e-04 - val_mse: 8.1979e-04 - val_mae: 0.0174
Epoch 12/180
65/65 [==============================] - 31s 483ms/step - loss: 0.0047 - mse: 0.0047 - mae: 0.0527 - val_loss: 8.5242e-04 - val_mse: 8.5242e-04 - val_mae: 0.0171
Epoch 13/180
65/65 [==============================] - 31s 474ms/step - loss: 0.0042 - mse: 0.0042 - mae: 0.0489 - val_loss: 8.5583e-04 - va

...

Epoch 79/180
65/65 [==============================] - 36s 554ms/step - loss: 9.3978e-04 - mse: 9.3978e-04 - mae: 0.0181 - val_loss: 9.8404e-04 - val_mse: 9.8404e-04 - val_mae: 0.0205
Epoch 80/180
65/65 [==============================] - 36s 547ms/step - loss: 9.4857e-04 - mse: 9.4857e-04 - mae: 0.0183 - val_loss: 9.9039e-04 - val_mse: 9.9039e-04 - val_mae: 0.0209
Epoch 81/180
65/65 [==============================] - 36s 551ms/step - loss: 9.5077e-04 - mse: 9.5077e-04 - mae: 0.0182 - val_loss: 9.7813e-04 - val_mse: 9.7813e-04 - val_mae: 0.0201
Epoch 82/180
65/65 [==============================] - 36s 553ms/step - loss: 9.5615e-04 - mse: 9.5615e-04 - mae: 0.0182 - val_loss: 9.8772e-04 - val_mse: 9.8772e-04 - val_mae: 0.0207
Epoch 83/180
65/65 [==============================] - 46s 702ms/step - loss: 9.4084e-04 - mse: 9.4084e-04 - mae: 0.0180 - val_loss: 9.7605e-04 - val_mse: 9.7605e-04 - val_mae: 0.0204
Epoch 84/180
65/65 [==============================] - 45s 688ms/step - loss: 9.3499e-

Epoch 124/180
65/65 [==============================] - 37s 568ms/step - loss: 8.4519e-04 - mse: 8.4519e-04 - mae: 0.0165 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0191
Epoch 125/180
65/65 [==============================] - 35s 538ms/step - loss: 8.4211e-04 - mse: 8.4211e-04 - mae: 0.0166 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0232
Epoch 126/180
65/65 [==============================] - 36s 554ms/step - loss: 8.4744e-04 - mse: 8.4744e-04 - mae: 0.0165 - val_loss: 9.7986e-04 - val_mse: 9.7986e-04 - val_mae: 0.0200
Epoch 127/180
65/65 [==============================] - 37s 569ms/step - loss: 8.4824e-04 - mse: 8.4824e-04 - mae: 0.0166 - val_loss: 9.3790e-04 - val_mse: 9.3790e-04 - val_mae: 0.0194
Epoch 128/180
65/65 [==============================] - 35s 538ms/step - loss: 8.4663e-04 - mse: 8.4663e-04 - mae: 0.0166 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0209
Epoch 129/180
65/65 [==============================] - 36s 552ms/step - loss: 8.4129e-04 - mse: 8.4129e-

Epoch 169/180
65/65 [==============================] - 36s 560ms/step - loss: 7.6570e-04 - mse: 7.6570e-04 - mae: 0.0154 - val_loss: 9.5425e-04 - val_mse: 9.5425e-04 - val_mae: 0.0188
Epoch 170/180
65/65 [==============================] - 37s 572ms/step - loss: 7.7667e-04 - mse: 7.7667e-04 - mae: 0.0156 - val_loss: 9.1054e-04 - val_mse: 9.1054e-04 - val_mae: 0.0180
Epoch 171/180
65/65 [==============================] - 36s 547ms/step - loss: 7.7122e-04 - mse: 7.7122e-04 - mae: 0.0153 - val_loss: 9.0613e-04 - val_mse: 9.0613e-04 - val_mae: 0.0189
Epoch 172/180
65/65 [==============================] - 39s 600ms/step - loss: 7.5594e-04 - mse: 7.5594e-04 - mae: 0.0155 - val_loss: 8.8163e-04 - val_mse: 8.8163e-04 - val_mae: 0.0169
Epoch 173/180
65/65 [==============================] - 35s 546ms/step - loss: 7.7389e-04 - mse: 7.7389e-04 - mae: 0.0155 - val_loss: 9.7020e-04 - val_mse: 9.7020e-04 - val_mae: 0.0199
Epoch 174/180
65/65 [==============================] - 35s 545ms/step - loss: 7.

65/65 [==============================] - 88s 1s/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0269 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0211
Epoch 38/180
65/65 [==============================] - 90s 1s/step - loss: 0.0015 - mse: 0.0015 - mae: 0.0263 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0220
Epoch 39/180
65/65 [==============================] - 85s 1s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0258 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0204
Epoch 40/180
65/65 [==============================] - 90s 1s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0249 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0196
Epoch 41/180
65/65 [==============================] - 89s 1s/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0249 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0203
Epoch 42/180
65/65 [==============================] - 89s 1s/step - loss: 0.0013 - mse: 0.0013 - mae: 0.0242 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0206
Epoch 43/180
65/65 [=========

65/65 [==============================] - 78s 1s/step - loss: 9.5039e-04 - mse: 9.5039e-04 - mae: 0.0182 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0197
Epoch 87/180
65/65 [==============================] - 77s 1s/step - loss: 9.5144e-04 - mse: 9.5144e-04 - mae: 0.0181 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0187
Epoch 88/180
65/65 [==============================] - 81s 1s/step - loss: 9.4869e-04 - mse: 9.4869e-04 - mae: 0.0182 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0190
Epoch 89/180
65/65 [==============================] - 81s 1s/step - loss: 9.4426e-04 - mse: 9.4426e-04 - mae: 0.0181 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0190
Epoch 90/180
65/65 [==============================] - 93s 1s/step - loss: 9.3636e-04 - mse: 9.3636e-04 - mae: 0.0180 - val_loss: 0.0011 - val_mse: 0.0011 - val_mae: 0.0189
Epoch 91/180
65/65 [==============================] - 117s 2s/step - loss: 9.3703e-04 - mse: 9.3703e-04 - mae: 0.0180 - val_loss: 0.0010 - val_mse: 0.001

In [ ]:
results = pd.DataFrame({'all_param':all_param,
                        
                        'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

results.to_csv('Results_LSTM_2_n_neurons_n_hidden.csv')
results

In [ ]:
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})

def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)